## __Analysis of DSC in multiple settings__

In [14]:
import numpy as np
import pandas as pd
import os
import glob
import shutil

import nibabel as nib
from sklearn.model_selection import train_test_split


from monai.apps import download_and_extract
from monai.apps.auto3dseg import (
    DataAnalyzer,
    BundleGen,
    AlgoEnsembleBestN,
    AlgoEnsembleBuilder,
    export_bundle_algo_history,
    import_bundle_algo_history,
)
from monai.auto3dseg import algo_to_pickle
from monai.bundle.config_parser import ConfigParser
from monai.config import print_config
from monai.utils.enums import AlgoKeys

In [1]:
def calculate_slice_all_labels_dice_score(segmentation, truth, label):
    segmentation = np.where(segmentation == label, 1, 0)
    truth = np.where(truth == label, 1, 0)
    area_sum = np.sum(segmentation) + np.sum(truth)
    if area_sum > 0:
        return np.sum(segmentation[truth==1])*2.0 / area_sum
    else:
        return 1
    
def calculate_nifti_all_labels_dice_score(seg_data, truth_data, label):
    z_range = range(seg_data.shape[-1])
    z_len = len(z_range)
    dice_sum = 0
    for z in  z_range:
        seg_slice = seg_data[:,:,z]
        truth_slice = truth_data[:,:,z]
        slice_dice = calculate_slice_all_labels_dice_score(seg_slice, truth_slice, label)
        dice_sum+=slice_dice

    return dice_sum / z_len

### __Cross Validation Instances__

In [80]:
import json

pred_folder_2d = "<PATH_TO_PRED_2D>"
pred_folder_2d_cv = "<PATH_TO_PRED_2D_CV>"

pred_folder_3d = "<PATH_TO_PRED_3D>"
pred_folder_3d_cv = "<PATH_TO_PRED_3D_CV>"

pred_folder_ensemble = "<PATH_TO_PRED_ENSEMBLE>"
true_folder_tr = "<PATH_TO_TRUE_LABELS_TR>"
true_folder_test = "<PATH_TO_TRUE_LABELS_TS>"
TEST_df = pd.read_csv("<PATH_TO_TEST_CSV>")

cv_datasplit_file = "<PATH_TO_CV_SPLITS_JSON>"

with open(cv_datasplit_file, "r") as f:
    cv_datasplit = json.load(f)

In [46]:
len(cv_datasplit)

5

In [68]:
import shutil
for idx, split in enumerate(cv_datasplit):
    cur_fold = "fold_%d" % idx
    dst = os.path.join("<PATH_TO_IMAGES_BY_FOLD_BASE>", "images_%s" % cur_fold)
    os.makedirs(dst, exist_ok=True)
    for case_id in split["val"]:
        fname = case_id + "_0000.nii.gz"
        src_fpath = os.path.join("<PATH_TO_IMAGES_TR>", case_id+"_0000.nii.gz")
        dst_fpath = os.path.join(dst, fname)
        shutil.copyfile(src_fpath, dst_fpath)
    # break
        

In [85]:
def compute_dice_cv(true_folder, pred_folder):
    result_dict = {}
    for idx, split in enumerate(cv_datasplit):
        dice_scores1, dice_scores2 = [], []
        for case_id in split["val"]:
            fname = case_id + ".nii.gz"
            cv_pred_folder = os.path.join(pred_folder, "fold_%d/validation" % idx)
            true_seg = nib.load(os.path.join(true_folder, fname)).get_fdata()
            pred_seg = nib.load(os.path.join(cv_pred_folder, fname)).get_fdata()
            dce1 = calculate_nifti_all_labels_dice_score(pred_seg, true_seg, label=1)
            dice_scores1.append(dce1)
            dce2 = calculate_nifti_all_labels_dice_score(pred_seg, true_seg, label=2)
            dice_scores2.append(dce2)
        result_dict["fold_%d_vb" % idx] = dice_scores1
        result_dict["fold_%d_ivd" % idx] = dice_scores2
    return result_dict


def compute_dice_cv_ensemble(true_folder, pred_folder):
    result_dict = {}
    for idx, split in enumerate(cv_datasplit):
        dice_scores1, dice_scores2 = [], []
        for case_id in split["val"]:
            fname = case_id + ".nii.gz"
            cv_pred_folder = os.path.join(pred_folder, "predict_ensemble_fold_%d" % idx)
            true_seg = nib.load(os.path.join(true_folder, fname)).get_fdata()
            pred_seg = nib.load(os.path.join(cv_pred_folder, fname)).get_fdata()
            dce1 = calculate_nifti_all_labels_dice_score(pred_seg, true_seg, label=1)
            dice_scores1.append(dce1)
            dce2 = calculate_nifti_all_labels_dice_score(pred_seg, true_seg, label=2)
            dice_scores2.append(dce2)
        result_dict["fold_%d_vb" % idx] = dice_scores1
        result_dict["fold_%d_ivd" % idx] = dice_scores2
        # break
    return result_dict

In [61]:
result_2d = compute_dice_cv(true_folder_tr, pred_folder_2d_cv)

In [64]:
result_3d = compute_dice_cv(true_folder_tr, pred_folder_3d_cv)

In [86]:
result_ens = compute_dice_cv_ensemble(true_folder_tr, pred_folder_ensemble)

In [88]:
for i in range(5):
    cur_fold = "fold_%d" % i
    print(f"\n>>>>>> {cur_fold}")
    print("--- 2D --- ")
    dice_scores1 = result_2d[cur_fold+"_vb"]
    dice_scores2 = result_2d[cur_fold+"_ivd"]
    print(f"Mean DCE for Vertebral body: {np.mean(dice_scores1):.3f} SD: {np.std(dice_scores1):.3f}, median: {np.median(dice_scores1):.3f}")
    print(f"Mean DCE for Vertebral discs: {np.mean(dice_scores2):.3f} SD: {np.std(dice_scores2):.3f}, median: {np.median(dice_scores2):.3f}")
    print(f"Macro-average DCE: {np.mean([np.mean(dice_scores1), np.mean(dice_scores2)]):.3f}, SD: {np.std([np.mean(dice_scores1), np.mean(dice_scores2)]):.3f}")
    
    print("--- 3D --- ")
    dice_scores1 = result_3d[cur_fold+"_vb"]
    dice_scores2 = result_3d[cur_fold+"_ivd"]
    print(f"Mean DCE for Vertebral body: {np.mean(dice_scores1):.3f} SD: {np.std(dice_scores1):.3f}, median: {np.median(dice_scores1):.3f}")
    print(f"Mean DCE for Vertebral discs: {np.mean(dice_scores2):.3f} SD: {np.std(dice_scores2):.3f}, median: {np.median(dice_scores2):.3f}")
    print(f"Macro-average DCE: {np.mean([np.mean(dice_scores1), np.mean(dice_scores2)]):.3f}, SD: {np.std([np.mean(dice_scores1), np.mean(dice_scores2)]):.3f}")
    
    print("--- Ensemble --- ")
    dice_scores1 = result_ens[cur_fold+"_vb"]
    dice_scores2 = result_ens[cur_fold+"_ivd"]
    print(f"Mean DCE for Vertebral body: {np.mean(dice_scores1):.3f} SD: {np.std(dice_scores1):.3f}, median: {np.median(dice_scores1):.3f}")
    print(f"Mean DCE for Vertebral discs: {np.mean(dice_scores2):.3f} SD: {np.std(dice_scores2):.3f}, median: {np.median(dice_scores2):.3f}")
    print(f"Macro-average DCE: {np.mean([np.mean(dice_scores1), np.mean(dice_scores2)]):.3f}, SD: {np.std([np.mean(dice_scores1), np.mean(dice_scores2)]):.3f}")
    
    
    # break
    


>>>>>> fold_0
--- 2D --- 
Mean DCE for Vertebral body: 0.921 SD: 0.088, median: 0.942
Mean DCE for Vertebral discs: 0.897 SD: 0.059, median: 0.915
Macro-average DCE: 0.909, SD: 0.012
--- 3D --- 
Mean DCE for Vertebral body: 0.908 SD: 0.099, median: 0.934
Mean DCE for Vertebral discs: 0.891 SD: 0.063, median: 0.910
Macro-average DCE: 0.900, SD: 0.009
--- Ensemble --- 
Mean DCE for Vertebral body: 0.921 SD: 0.089, median: 0.949
Mean DCE for Vertebral discs: 0.897 SD: 0.057, median: 0.913
Macro-average DCE: 0.909, SD: 0.012

>>>>>> fold_1
--- 2D --- 
Mean DCE for Vertebral body: 0.911 SD: 0.077, median: 0.942
Mean DCE for Vertebral discs: 0.891 SD: 0.067, median: 0.912
Macro-average DCE: 0.901, SD: 0.010
--- 3D --- 
Mean DCE for Vertebral body: 0.899 SD: 0.082, median: 0.911
Mean DCE for Vertebral discs: 0.884 SD: 0.073, median: 0.907
Macro-average DCE: 0.892, SD: 0.007
--- Ensemble --- 
Mean DCE for Vertebral body: 0.909 SD: 0.079, median: 0.937
Mean DCE for Vertebral discs: 0.892 SD: 0

### __Test Set__

In [40]:
def compute_dice_test(true_folder, pred_folder, TEST_df):
    N_test = len(TEST_df)
    dice_scores1 = []
    dice_scores2 = []
    for i in range(N_test):
        fname = "cspine_%d.nii.gz" % i
        true_seg = nib.load(os.path.join(true_folder, fname)).get_fdata()
        pred_seg = nib.load(os.path.join(pred_folder, fname)).get_fdata()
        dce1 = calculate_nifti_all_labels_dice_score(pred_seg, true_seg, label=1)
        dice_scores1.append(dce1)
        dce2 = calculate_nifti_all_labels_dice_score(pred_seg, true_seg, label=2)
        dice_scores2.append(dce2)
    print(f"Mean DCE for Vertebral body: {np.mean(dice_scores1)} SD: {np.std(dice_scores1)}, median: {np.median(dice_scores1)}")
    print(f"Mean DCE for Vertebral discs: {np.mean(dice_scores2)} SD: {np.std(dice_scores2)}, median: {np.median(dice_scores2)}")
    print(f"Macro-average DCE: {np.mean([np.mean(dice_scores1), np.mean(dice_scores2)])}, SD: {np.std([np.mean(dice_scores1), np.mean(dice_scores2)])}")
    
    return dice_scores1, dice_scores2

In [41]:
from scipy.stats import mannwhitneyu, bootstrap

In [42]:
print(f"\n Dice for 2d ---")
vb_2d, ivd_2d = compute_dice_test(true_folder_test, pred_folder_2d, TEST_df)
print(f"\n Dice for 3d full res ---")
vb_3d, ivd_3d = compute_dice_test(true_folder_test, pred_folder_3d, TEST_df)
print(f"\n Dice for ensemble ---")
vb_ens, ivd_ens = compute_dice_test(true_folder_test, pred_folder_ensemble, TEST_df)


 Dice for 2d ---
Mean DCE for Vertebral body: 0.928301454654587 SD: 0.049685697155445525, median: 0.9509430016129665
Mean DCE for Vertebral discs: 0.9014971958375567 SD: 0.04662977311107608, median: 0.915840542669298
Macro-average DCE: 0.9148993252460719, SD: 0.013402129408515151

 Dice for 3d full res ---
Mean DCE for Vertebral body: 0.9250185060397667 SD: 0.046696470683088916, median: 0.9433601006143835
Mean DCE for Vertebral discs: 0.8998041299182035 SD: 0.0473950086236868, median: 0.912927111677251
Macro-average DCE: 0.912411317978985, SD: 0.012607188060781593

 Dice for ensemble ---
Mean DCE for Vertebral body: 0.9287320875644387 SD: 0.04750183290156479, median: 0.9499220156830561
Mean DCE for Vertebral discs: 0.9038726755363976 SD: 0.04529133062341535, median: 0.9165467049265648
Macro-average DCE: 0.9163023815504181, SD: 0.012429706014020547


In [37]:
def mu_test_ci(score1, score2):
    _, pval = mannwhitneyu(score1, score2)
    bootstrap_ci = bootstrap((score1, score2), median_diff, confidence_level=0.95, random_state=42, method="percentile", vectorized=False)
    print(f"p-value: {pval:.3f}, confidence interval: [{bootstrap_ci.confidence_interval.low}, {bootstrap_ci.confidence_interval.high}]")
    # return pval, (bootstrap_ci.confidence_interval.low, bootstrap_ci.confidence_interval.high)

In [39]:
print(f"\n comparison 2d vs ens on vertebral bodies:")
mu_test_ci(vb_2d, vb_ens)
print(f"\n comparison 2d vs ens on ivd:")
mu_test_ci(ivd_2d, ivd_ens)

print(f"\n comparison 3d vs ens on vertebral bodies:")
mu_test_ci(vb_3d, vb_ens)
print(f"\n comparison 3d vs ens on ivd:")
mu_test_ci(ivd_3d, ivd_ens)


 comparison 2d vs ens on vertebral bodies:
p-value: 0.925, confidence interval: [-0.014820120061783059, 0.017307315637482012]

 comparison 2d vs ens on ivd:
p-value: 0.688, confidence interval: [-0.021265059617154185, 0.017763017972567446]

 comparison 3d vs ens on vertebral bodies:
p-value: 0.392, confidence interval: [-0.03238331706109798, 0.01212200694073813]

 comparison 3d vs ens on ivd:
p-value: 0.565, confidence interval: [-0.01984789499242923, 0.01706011240769436]
